In [ ]:
#
#    Notebook de cours MAP412 - Chapitre 2 - M. Massot 2022-2023 - Ecole polytechnique
#    ----------   
#    Distribution des points et conditionnement
#    
#    Auteurs : L. Séries et M. Massot - (C) 2022
#    

# Distribution des points et conditionnement

In [ ]:
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.io as pio
pio.templates.default = "seaborn"

def improved_lag_interp(xk, yk, x):
    n = xk.size
    l = np.ones(x.size)
    p = np.zeros(x.size)
    
    wk = np.zeros(n) 
    for k in range(n):
        wk[k] = 1. / (np.prod( xk[k] - xk[np.arange(n)!=k] ))
    
    mask = np.in1d(x, xk[:])
    p[np.where(mask)] = yk[np.where(np.in1d(xk, x))]
    mask = np.invert(mask)    

    for k in range(n):
        l[mask] *= x[mask] - xk[k]
        p[mask] += (wk[k]/(x[mask] - xk[k])) * yk[k]
        
    return l*p

def cheb_points(xmin, xmax, n):
    x = np.zeros(n+1)
    for i in range(n+1):
        x[i] = (xmin+xmax)/2 + ((xmax-xmin)/2) * np.cos(((2*i+1)*np.pi)/(2*n + 2))
    return x

def noise(eps, x): 
    return eps * (2*np.random.rand(x.size)-1) 

def lagrange(k, xk, x):
    lag = np.ones(x.size)
    for j in range(xk.size):
        if (j!=k): lag *= (x-xk[j])/(xk[k]-xk[j])
    return lag

def compute_cond(xk, yk, pn, x):
    cond = np.zeros(x.size)
    for k in range(xk.size):
        cond += np.abs(lagrange(k, xk, x) * yk[k])
    return cond / np.abs(pn)

On considère l’approximation polynomiale de la fonction $f(x)=\sin(\pi x)$ sur $[-1,1]$.

In [ ]:
def f(x):
    return np.sin(np.pi*x)

xmin = -1.; xmax = 1.

On utilise la première forme de la formule d’interpolation barycentrique qui présente de bonnes propriétés de stabilité. 

## Points équidistribués

On interpole la fonction $f(x)$ en utilisant des points équidistants.  

In [ ]:
# x use to display f and pn
xplot = np.linspace(xmin, xmax, 500)

# x use to display relative conditionning
eps = 0.05
xcond = np.concatenate((np.linspace(xmin+eps, -eps, 50), np.linspace(eps, xmax-eps, 50) ))

# maximal degree of Newton interpolating polynomial
n_max = 72

# array of degree
n = np.arange(19, n_max+1, 2)

# compute for each degree xk, yk and pn
xk = []; yk = []; pn = []; cond = []
for i, ni in enumerate(n):
    xk.append(np.linspace(xmin, xmax, ni+1))
    yk.append(f(xk[i]))
    pn.append(improved_lag_interp(xk[i], yk[i], xplot))
    cond.append(compute_cond(xk[i], xk[i], improved_lag_interp(xk[i], yk[i], xcond), xcond))

# Create figure
marker=dict(size=5, symbol='x-thin', line=dict(width=1, color='rgb(76,114,176)'))
fig = make_subplots(rows=2, cols=1, vertical_spacing = 0.1,
                    subplot_titles=("Approximation polynomiale", "Conditionnement relatif local"))

# add f(x) plot
fig.add_trace(go.Scatter(visible=True, x=xplot, y=f(xplot), name="f(x)", legendgroup='1'))

# add yi and pn(x) invisible plots
for i, ni in enumerate(n):
    fig.add_trace(go.Scatter(visible=False, x=xplot, y=pn[i], name=f"p{ni}(x)"))
    fig.add_trace(go.Scatter(visible=False, x=xk[i], y=yk[i], mode='markers', name="pts d'interpolation"))
    fig.add_trace(go.Scatter(visible=False, x=xcond, y=cond[i], mode='markers', showlegend=False, marker=marker), row=2, col=1)

# Make plot visible for n[0]
fig.data[1].visible = True
fig.data[2].visible = True
fig.data[3].visible = True

# Create and add slider
steps = []
for i, ni in enumerate(n):
    step = dict(method="update", label = f" {ni+1}",
                args=[{"visible": [el==0 or el==3*i+1 or el==3*i+2 or el==3*i+3 for el in range(len(fig.data))]}])
    steps.append(step)
        
sliders = [dict(currentvalue={"prefix": "nb points: "}, steps=steps)]

legend = dict(yanchor="top", y=0.99, xanchor="left", x=0.10, bgcolor='rgba(0,0,0,0)')
title = "Approximation polynomiale utilisant des points équidistants"
fig.update_layout(sliders=sliders, height=750, legend=legend, title=title)
fig.update_yaxes(range=[-1.1, 1.1], row=1)
fig.update_yaxes(type="log", exponentformat='e', row=2)

fig.show()

Pour cette distribution de points, on observe des erreurs conséquentes dès que le nombre de points dépasse 60. L'algorithme utilisé étant stable, les erreurs constatées sont directement liées au conditionnement local du problème qui augmente avec le nombre de points.

## Points de Tchebychev

On interpole la fonction $f(x)$ en utilisant des points de Tchebychev.  

In [ ]:
# x use to display f and pn
xplot = np.linspace(xmin, xmax, 1500)

# x use to display relative conditionning
eps = 0.05
xcond = np.concatenate((np.linspace(xmin+eps, -eps, 50), np.linspace(eps, xmax-eps, 50) ))
#xcond = np.linspace(xmin+0.1, xmax-0.1, 100)

# maximal degree of Newton interpolating polynomial
n_max = 520

# array of degree
n = np.arange(19, n_max+1, 40)

# compute for each degree xk, yk and pn
xk = []; yk = []; pn = []; cond = []
for i, ni in enumerate(n):
    xk.append(cheb_points(xmin, xmax, ni))
    yk.append(f(xk[i]))
    pn.append(improved_lag_interp(xk[i], yk[i], xplot))
    cond.append(compute_cond(xk[i], xk[i], improved_lag_interp(xk[i], yk[i], xcond), xcond))

# Create figure
marker=dict(size=5, symbol='x-thin', line=dict(width=1, color='rgb(76,114,176)'))
fig = make_subplots(rows=2, cols=1, vertical_spacing = 0.1,
                    subplot_titles=("Approximation polynomiale", "Conditionnement relatif local"))

# add f(x) plot
fig.add_trace(go.Scatter(visible=True, x=xplot, y=f(xplot), name="f(x)", legendgroup='1'))

# add yi and pn(x) invisible plots
for i, ni in enumerate(n):
    fig.add_trace(go.Scatter(visible=False, x=xplot, y=pn[i], name=f"p{ni}(x)"))
    fig.add_trace(go.Scatter(visible=False, x=xk[i], y=yk[i], mode='markers', name="pts d'interpolation"))
    fig.add_trace(go.Scatter(visible=False, x=xcond, y=cond[i], mode='markers', showlegend=False, marker=marker), row=2, col=1)

# Make plot visible for n[0]
fig.data[1].visible = True
fig.data[2].visible = True
fig.data[3].visible = True

# Create and add slider
steps = []
for i, ni in enumerate(n):
    step = dict(method="update", label = f" {ni+1}",
                args=[{"visible": [el==0 or el==3*i+1 or el==3*i+2 or el==3*i+3 for el in range(len(fig.data))]}])
    steps.append(step)
        
sliders = [dict(currentvalue={"prefix": "nb points: "}, steps=steps)]

legend = dict(yanchor="top", y=0.99, xanchor="left", x=0.10, bgcolor='rgba(0,0,0,0)')
title = "Approximation polynomiale utilisant des points de Tchebychev"
fig.update_layout(sliders=sliders, height=750, legend=legend, title=title)
fig.update_yaxes(range=[-1.1, 1.1], row=1)

fig.show()

Pour cette distribution de points, on peut augmenter les nombre de points sans observer d'erreur. C'est le comportement attendu de l'utilisation d'un algorithme stable pour un bon conditionnement local du problème.